# TensorFlow Keras MNIST 분류 - 로컬 예제

_**(하위 집합인) [MNIST DIGITS](https://en.wikipedia.org/wiki/MNIST_database) 데이터 세트에 대한 TF.Keras CNN 분류기를 학습하고 내보냅니다: 노트북에서 모든 저장 및 계산을 로컬로 수행합니다**_.

이 노트북은 SageMaker 스튜디오의 `Python 3 (TensorFlow 2.3 Python 3.7 CPU Optimized)` 커널 또는 클래식 SageMaker 노트북 인스턴스의 `conda_tensorflow2_p37`에서 잘 작동합니다.

---

[데이터셋](https://s3.amazonaws.com/fast-ai-imageclas/mnist_png.tgz)은 [AWS의 오픈 데이터 레지스트리](https://registry.opendata.aws/fast-ai-imageclas/)에서 호스팅되며, 각 숫자가 나타내는 폴더에 정리된 PNG 이미지가 포함되어 있습니다.

>❓이 노트북의 워크플로우를 SageMaker를 사용하여 더 효과적으로 다시 만드는 방법을 알아낼 수 있습니까?

## 내용

1. **[노트북 설정](#Notebook-Setup)**
1. **[데이터 준비](#Prepare-the-Data)**
1. **[파일에서 데이터 로드](#Load-the-Data-From-File)**
1. **[CNN을 위한 데이터 사전 처리](#Pre-Process-the-Data-for-our-CNN)**
1. **[모델 구축](#Build-a-Model)**
1. **[모델 학습](#Fit-the-Model)**
1. **[학습 모델 저장](#Save-the-Trained-Model)**
1. **[결과 탐색](#Explore-Results)**

자세한 지침은 함께 제공되는 **Instructions** 노트를 참조하세요!

## 노트북 설정

평소처럼 필요한 라이브러리를 추가로 설치하고 종속 요소를 가져오는 것으로 시작하겠습니다.

> ℹ️ **설치 문제 해결**을 참조하세요. 아래의 `ModuleNotFoundError`가 발생하거나 대화형 위젯이 렌더링되지 않는 경우:
>
> -  아래 `!pip install` 명령을 실행하고 노트북 커널을 재시작한 후 다른 코드 셀을 실행했는지 확인합니다.
> -  모듈이 성공적으로 설치된 것처럼 보이지만 노트북 커널 환경에서 누락된 경우, 모듈을 올바른 위치에 설치하려면 `%pip install`을 대신 실행해야 할 수 있습니다.
> - 특히 영향을 받는 모듈을 이미 `import`했거나 스튜디오가 아닌 SageMaker 노트북 인스턴스에서 실행 중인 경우, pip 라이브러리를 설치한 후 노트북 커널을 다시 시작해야 할 수 있습니다.
> - **`@jupyter-widgets/jupyterlab-manager`** 및 **`ipycanvas`** JupyterLab 위젯이 설치되어 있는지 확인합니다(퍼즐 조각 아이콘 "Extension Manager" 사이드바 탭을 사용하거나 보이지 않는 경우 *Settings > Enable Extension Manager*를 클릭합니다). 메시지가 표시되면 JupyterLab을 '재빌드'한 다음 작업을 저장하고 빌드가 완료되면 페이지를 새로고침합니다. Studio에서 시스템 터미널을 열고 `restart-jupyter-server`를 실행해야 할 수도 있습니다.
>
> 실제로 (그리고 이 워크샵의 CloudFormation 템플릿에서) JupyterLab 확장 프로그램은 일반적으로 사용자가 수동으로 설치하는 대신 [Studio](https://aws.amazon.com/blogs/machine-learning/customize-amazon-sagemaker-studio-using-lifecycle-configurations/) 또는 [노트북 인스턴스](https://docs.aws.amazon.com/sagemaker/latest/dg/notebook-lifecycle-config.html)에 대한 **라이프사이클 구성 스크립트**를 통해 설치됩니다.

In [ ]:
# First install some libraries which might not be available across all kernels:
!pip install "ipycanvas<0.13" "ipywidgets<8" matplotlib

> ⚠️ ***노트북 커널을 재시작***한 후 계속하세요! (도구 모음의 원형 화살표 버튼)

설치 후 커널을 재시작하지 않으면 나중에 대화형 그리기 위젯이 작동하지 않을 수 있습니다.

In [ ]:
%load_ext autoreload
%autoreload 2

# Python Built-Ins:
import glob
import os

# External Dependencies:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.models import Sequential

# Local Notebook Utils:
import util

%matplotlib inline

print(f"Using TensorFlow version {tf.__version__}")
print(f"Keras version {tf.keras.__version__}")

## 데이터 준비

이제 이미지 데이터를 다운로드해 보겠습니다.

원본 MNIST 데이터에는 7만 개의 작은 28x28픽셀 PNG 파일(학습 데이터 세트에 6만 개, 테스트 데이터 세트에 1만 개)이 있습니다. 이 형식은 익숙하고 좋은 형식이지만, 많은 수의 작은 파일은 저장과 전송에 비효율적이므로, **성능을 유지하기 위해** 다음과 같이 처리할 것입니다:

- 데이터를 `/tmp` 아래의 로컬 임시 폴더에 다운로드합니다(즉, SageMaker의 왼쪽 사이드바에 파일이 표시되지 않음).
- 작업할 데이터의 하위 집합만 샘플링합니다.

In [ ]:
local_dir = "/tmp/mnist"
training_dir = f"{local_dir}/training"
testing_dir = f"{local_dir}/testing"

# Download the MNIST data from the Registry of Open Data on AWS
!rm -rf {local_dir}
!mkdir -p {local_dir}
!aws s3 cp s3://fast-ai-imageclas/mnist_png.tgz {local_dir} --no-sign-request

# Un-tar the MNIST data, stripping the leading path element; this will leave us with directories
# {local_dir}/testing/ and {local_dir/training/
!tar zxf {local_dir}/mnist_png.tgz -C {local_dir}/ --strip-components=1 --no-same-owner

# Get the list of files in the training and testing directories recursively
train_files = sorted(list(glob.iglob(os.path.join(training_dir, "*/*.png"), recursive=True)))
test_files = sorted(list(glob.iglob(os.path.join(testing_dir, "*/*.png"), recursive=True)))

print(f"Training files: {len(train_files)}")
print(f"Testing files:  {len(test_files)}")

# Reduce the data by keeping every Nth file and dropping the rest of the files.
reduction_factor = 2
train_files_to_keep = train_files[::reduction_factor]
test_files_to_keep = test_files[::reduction_factor]

print(f"Training files kept: {len(train_files_to_keep)}")
print(f"Testing files kept:  {len(test_files_to_keep)}")

# Delete all the files not to be kept
for fname in set(train_files) ^ set(train_files_to_keep):
    os.remove(fname)

for fname in set(test_files) ^ set(test_files_to_keep):
    os.remove(fname)

print("Done!")

## 파일에서 데이터 로드

이제 이미지가 `{local_dir}` 폴더에 저장되었으므로 이 파일에서 학습 및 테스트 세트를 읽어와 보겠습니다.
```
    {local_dir}
    |----------------.
    `-- testing      `-- training
        |-- 0       |-- 0
        |               `-- 1.png
        |-- 1       |-- 1
        |-- 2       |-- 2
        |-- 3       |-- 3
        |-- 4       |-- 4
        |-- 5       |-- 5
        |-- 6       |-- 6
        |-- 7       |-- 7
        |-- 8       |-- 8
        `-- 9       `-- 9
```

(학습 및 테스트 모두) 폴더 이름에서 대상 레이블(`0`-`9`)을 가져와 각 폴더를 반복하고 각 PNG를 이미지 매트릭스에 로드합니다.

In [ ]:
from PIL import Image

labels = sorted(os.listdir(training_dir))
n_labels = len(labels)

x_train = []
y_train = []
x_test = []
y_test = []
print("Loading label ", end="")
for ix_label in range(n_labels):
    label_str = labels[ix_label]
    print(f"{label_str}...", end="")
    trainfiles = filter(
        lambda s: s.endswith(".png"),
        os.listdir(os.path.join(training_dir, label_str)),
    )

    for filename in trainfiles:
        # Can't just use tf.keras.preprocessing.image.load_img(), because it doesn't close its file
        # handles! So get "Too many open files" error... Grr
        with open(os.path.join(training_dir, label_str, filename), "rb") as imgfile:
            x_train.append(
                # Squeeze (drop) that extra channel dimension, to be consistent with prev format:
                np.squeeze(tf.keras.preprocessing.image.img_to_array(Image.open(imgfile)))
            )
            y_train.append(ix_label)

    # Repeat for test data:
    testfiles = filter(
        lambda s: s.endswith(".png"),
        os.listdir(os.path.join(testing_dir, label_str)),
    )

    for filename in testfiles:
        with open(os.path.join(testing_dir, label_str, filename), "rb") as imgfile:
            x_test.append(
                np.squeeze(tf.keras.preprocessing.image.img_to_array(Image.open(imgfile)))
            )
            y_test.append(ix_label)
print()


print("Shuffling trainset...")
train_shuffled = [(x_train[ix], y_train[ix]) for ix in range(len(y_train))]
np.random.shuffle(train_shuffled)

x_train = np.array([datum[0] for datum in train_shuffled])
y_train = np.array([datum[1] for datum in train_shuffled])
train_shuffled = None

print("Shuffling testset...")
test_shuffled = [(x_test[ix], y_test[ix]) for ix in range(len(y_test))]
np.random.shuffle(test_shuffled)

x_test = np.array([datum[0] for datum in test_shuffled])
y_test = np.array([datum[1] for datum in test_shuffled])
test_shuffled = None

print("Done!")

**계속 진행하기 전에**, 데이터 분포를 빠르게 시각화해 보겠습니다.

In [ ]:
print(f"x_train.shape {x_train.shape}; dtype {x_train.dtype}")
print(f"y_train.shape {y_train.shape}; dtype {y_train.dtype}")
print(f"x_test.shape {x_test.shape}; dtype {x_test.dtype}")
print(f"y_test.shape {y_test.shape}; dtype {y_test.dtype}")

fig = plt.figure(figsize=(14, 3))
ax = plt.subplot(1, 2, 1)
plt.hist(x_train.flatten())
ax.set_title("Histogram of Training Image Data")
ax.set_ylabel("Frequency in Training Set")
ax.set_xlabel("Pixel Value")

ax = plt.subplot(1, 2, 2)
plt.hist(y_train)
ax.set_title("Histogram of Training Set Labels")
ax.set_ylabel("Frequency in Training Set")
ax.set_xlabel("Y Label Value")

plt.show()

데이터는 레이블 0~9 사이에 꽤 고르게 분포되어 있으며, 이미지는 0에서 255까지의 고정 크기 28x28 행렬로 인코딩되어 있는 것처럼 보입니다. 여기서는 몇 가지 예시를 통해 이해를 돕도록 하겠습니다:

In [ ]:
print("Some example images:")
fig = plt.figure(figsize=(14, 2))
for i in range(5):
    fig = plt.subplot(1, 5, i + 1)
    ax = plt.imshow(x_train[i], cmap="gray")
    fig.set_title(f"Number {y_train[i]}")
plt.show()

## CNN을 위한 데이터 사전 처리하기

다음으로 신경망에 맞게 이 형식을 조정하겠습니다:

- 픽셀 값을 정규화하여 숫자 컨디셔닝을 개선합니다.
- 각 숫자에 대한 확률의 소프트맥스 분류기 출력에 맞게 레이블을 원핫 인코딩합니다.
- 배치 차원(여러 샘플을 병렬로 처리하기 위한)과 채널 차원(예: 흑백 단일 채널을 제외한 3채널 RGB 이미지와 같은)을 모두 추가하고 X축과 Y축을 추가합니다.

In [ ]:
# Since we're actually feeding the images in to nets this time, we should actually pay attention
# to which way around Keras wants the channel dimension:
if K.image_data_format() == "channels_first":
    x_train = np.expand_dims(x_train, 1)
    x_test = np.expand_dims(x_train, 1)
else:
    x_train = np.expand_dims(x_train, len(x_train.shape))
    x_test = np.expand_dims(x_test, len(x_test.shape))

x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
x_train /= 255
x_test /= 255

input_shape = x_train.shape[1:]

print("x_train shape:", x_train.shape)
print("input_shape:", input_shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, n_labels)
y_test = tf.keras.utils.to_categorical(y_test, n_labels)

print("n_labels:", n_labels)
print("y_train shape:", y_train.shape)

## 모델 구축

이 모델의 핵심은 가능한 모든 레이블에 대한 신뢰도 점수를 산출하는 소프트맥스 출력 레이어가 있는 2D 컨볼루션 네트워크입니다(예: 숫자 = 0~9에 대한 10가지 옵션).

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(n_labels, activation="softmax"))

model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer=tf.keras.optimizers.Adadelta(),
    metrics=["accuracy"],
)

## 모델 피팅(학습)

Keras를 사용하면 모델 피팅(학습)과 평가가 매우 간단합니다: 적절한 후크가 없어서 기본 로깅에 만족합니다:

In [ ]:
%%time
batch_size = 128
epochs = 12

model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    shuffle=True,
    verbose=1,  # Hint: You might prefer =2 for running in SageMaker!
    validation_data=(x_test, y_test),
)

score = model.evaluate(x_test, y_test, verbose=0)
print(f"Test loss={score[0]}")
print(f"Test accuracy={score[1]}")

## 학습된 모델 저장

Keras에는 `model.save()` 명령이 내장되어 있으며, TensorFlow v2에서는 이 명령으로 TensorFlow Serving과 호환되는 출력을 직접 생성할 수 있습니다!

...하지만 이 노트북은 TensorFlow v1을 실행합니다. 이 명령어를 알아내야 하는 번거로움을 덜어드리기 위해(이 주제에 대한 좋은 블로그 포스팅이 있습니다. [여기](https://aws.amazon.com/blogs/machine-learning/deploy-trained-keras-or-tensorflow-models-using-amazon-sagemaker/)), 여기서는 모델을 TensorFlow Serving-ready 형식으로 저장하여 힌트를 드리겠습니다.

In [ ]:
# The export folder needs to be empty, or non-existent
!rm -rf data/model/model/1

# Please ignore Layer.updates(...) warning if any while running the notebook in < TFv2.4
model.save(os.path.join("data/model", "model/1"))

## 결과 탐색

모델을 시험해 보기 위해 테스트 세트에서 샘플 이미지를 가져와서 레이블을 예측하고 플로팅할 수 있습니다:

In [ ]:
# Choose an image:
label = "3"
filename = os.listdir(f"{testing_dir}/{label}")[0]

# Load the image (and normalize to 0-1):
img = tf.keras.preprocessing.image.img_to_array(
    Image.open(f"{testing_dir}/{label}/{filename}")
) / 255

# Expand out the "batch" dimension, and send to the model:
result = model.predict(np.expand_dims(img, 0))
print(f"Result confidences: {result}")

# Plot the result:
plt.figure(figsize=(3, 3))
fig = plt.subplot(1, 1, 1)
ax = plt.imshow(np.squeeze(img), cmap="gray")
fig.set_title(f"Predicted Number {np.argmax(result[0])}")
plt.show()

모두 완료되었습니다!